# Access to Swisstopo Lidar Data

This notebook shows and explain how to get the low resolution Lidar data from Swisstopo and how to process them.

Most example are taken or inspired from [laspy documentation](https://laspy.readthedocs.io/en/latest/index.html).

In [ ]:
# These library are needed for the notebook
!pip install numpy          # must have
!pip install pandas         # must have
!pip install laspy          # must have

!pip install matplotlib     # nice to have

## Get the data

Go to [this](https://www.swisstopo.admin.ch/fr/geodata/height/surface3d.html#technische_details) website, to the 'swissSURFACE3D - Accès aux géodonnées' part and select the area of data that you want. 

For this example we select by commune and selct the commune of Fribourg (FR). As option, we keep the format LIDAR (ZIP), coordonates system MN95 (default) and the actual state. 
4 links are made available to download. For the sake of the example, we only download the first one here: 'swisssurface3d_2019_2576-1183_2056_5728.las.zip'. Click on 'Download'.

You now have a file 'swisssurface3d_2019_2576-1183_2056_5728.las.zip' locally of around 130MB (or any other las.zip file). Make it accessible from your notebook.

## Process the data

In [ ]:
# TODO: adapt this path to access the downloaded data from your notebook
SOURCE_LIDAR_ZIP = 'swisssurface3d_2019_2576-1183_2056_5728.las.zip'
TARGET_LIDAR_FOLDER = 'fribourg_part_1' 

### Unzip the file and get its path

In [ ]:
import zipfile

with zipfile.ZipFile(SOURCE_LIDAR_ZIP, 'r') as zip_ref:
    zip_ref.extractall(TARGET_LIDAR_FOLDER)

In [ ]:
import os
las_path = os.path.join(TARGET_LIDAR_FOLDER, os.listdir(TARGET_LIDAR_FOLDER)[0])
las_path

### Open Lidar data
The file is stored in [las format](https://laspy.readthedocs.io/en/latest/intro.html) which is a public standard format for exchanging 3D point data. There are different python librairies to process this type of data. Here we will use [laspy](https://pypi.org/project/laspy/).

In [ ]:
import laspy as lp
import numpy as np

### Get metadata information

In [ ]:
#  Read and show metadate about point cloud
with lp.open(las_path) as f:
    # Get metadata information
    print(f"Point format:             {f.header.point_format}")
    print(f"Number of points:         {f.header.point_count}")
    print(f"Variable length record:   {f.header.vlrs}\n")
    
    # Read the point cloud data
    las = f.read()
    print(f"Las file:                 {las}")
    point_format = las.point_format
    print(f"Dimensions:               {list(point_format.dimension_names)}\n")
    
    # Offset and scaling for projections (discussed later)
    print(f"Points offset:            {f.header.offset}")
    print(f"Points scaling:           {f.header.scale}")

### Process Lidar Data

In [ ]:
import pandas as pd

In [ ]:
las = lp.read(las_path)

In [ ]:
# Get pandas dataset of the point cloud coordinates and class
df = pd.DataFrame({'X': las.X, 'Y': las.Y, 'Z':las.Z, 'Class': np.array(las.classification)})
df.head()

From there, you can do anything that you can usually do in pandas. You can modify the dataset to add other columns from the point cloud.

### Plot Lidar data

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Subsample (to have less points)
print(f'Previous size: {df.shape}')
df = df.sample(frac=0.50, replace=False, random_state=1)

# Keep a smaller region
df = df[df['X'] > np.mean(df['X'])]
df = df[df['Y'] > np.mean(df['Y'])]
print(f'New size: {df.shape}')

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = plt.axes(projection='3d')
ax.scatter3D(df['X'], df['Y'], df['Z'], c =  df['Class'], cmap='viridis', s=1);

### Reproject to another Reference Coordinate System

Unfortunatelly with these data, the variable length record which should contain the reference coordinate system (see above in metadata part) is empty. 

Based on the additonal documentation of swissSURFACE3D, we know that every downloaded tiles names contain an id which corresponds to the kilometric coordinates in the reference system [MN95] (https://www.swisstopo.admin.ch/fr/connaissances-faits/mensuration-geodesie/cadres-de-reference/local/mn95.html#234_1607609199106) in the south-west angle.

Hence our `swisssurface3d_2019_2576-1183_2056_5728.las` would be the tile recorded in 2019 at the coordinate (2576, 1183) as can be checked below:

In [ ]:
#  Read and show metadate about point cloud
with lp.open(las_path) as f:
    # Offset and scaling for projections ()
    print(f"Points offset:            {f.header.offset}")
    print(f"Points scaling:           {f.header.scale}")
    x_offset, y_offset, z_offset = f.header.offset
    x_scale, y_scale, z_scale = f.header.scale

To reconstruct the original point cloud in MN95 coordinates in the system CH1903+, the coordinates are calculated using the following formula(s): \
x = (x_int * x_scale) + x_offset \
y = (y_int * y_scale) + y_offset \
z = (z_int * z_scale) + z_offset

In [ ]:
df['X'] = df['X'] * x_scale + x_offset
df['Y'] = df['Y'] * y_scale + y_offset
df['Z'] = df['Z'] * z_scale + z_offset

In [ ]:
df.rename(columns = {'X': 'Easting', 'Y': 'Northing', 'Z': 'Elevation'}, inplace = True)

In [ ]:
df.head() # in CH1903+ coordinates

To reproject in any other coordinate reference system, more complicated formulas are required. They can be found online and applied to the points.